# LGIS exercise

Given: A positive integer n≤10000 followed by a permutation π of length n.

Return: A longest increasing subsequence of π, followed by a longest decreasing subsequence of π.

In [ ]:
# we want to find the longest increasing sequence (LIS)
def LISEQ(seq):
    n = len(seq)
    table = [1] * n # I create this table to store the length of our LIS, 
                    # I put all value equal to 1 cause the LIS at each index is at least the element itself
    pred = [-1] * n # this array is needed to trace back the sequence to reconstruct it
                    # it keeps track of which element in the sequence came just before a given element in the LIS
                    # -1 cause the first element has no predecessor

    # now we calculate the LIS using dynamic programming
    for i in range(n):
        for j in range(i):
            if seq[j] < seq[i] and table[j] + 1 > table[i]: # this means that if seq[j] < seq[i], seq[i] can extend the LIS ending at seq[j]
                                                            # then it checks if extending the sequence gives a longer LIS
                table[i] = table[j] + 1 # we update the LIS length 
                pred[i] = j             # and we update the pred as well

    # now we need to reconstruct the LIS
    lis = []
    idx = table.index(max(table)) # we find the index of the max LIS length
    while idx != -1:              # the while loop assures that we trace back till there are no predecessors
        lis.append(seq[idx])      # we add the element to the LIS
        idx = pred[idx]           # and update to move to the predecessor of the current element

    return lis[::-1] # now we need to reverse the LIS since it was constructed backwards

# now we want to find the longest decreasing sequence (LDS)
def LDSEQ(seq):
    return LISEQ(seq[::-1])[::-1]  # to do this, we reverse the input sequence, then find the LIS of the reversed sequence
                                   # and then reverse the result back to get the LDS

def filereader(input_file):
    with open(input_file, 'r') as file:
        n = int(file.readline().strip())  
        perm = list(map(int, file.readline().strip().split()))
    return perm

input_file = 'rosalind_lgis.txt'  
perm = filereader(input_file)
result1 = LISEQ(perm)  # the longest increasing sequence
result2 = LDSEQ(perm)  # the longest decreasing sequence

print(" ".join(map(str, result1)))
print(" ".join(map(str, result2)))

# SSEQ exercise

Given: Two DNA strings s and t (each of length at most 1 kbp) in FASTA format.
 
Return: One collection of indices of s in which the symbols of t appear as a subsequence of s.

In [ ]:
from Bio import SeqIO

def SSEQ(input_file):
    records = list(SeqIO.parse(input_file, "fasta"))
    s = str(records[0].seq)  
    t = str(records[1].seq)  # this is the substring we want to find in s

    position = 0 # I inizialize the starting position to search in the sequence s
    indices = [] # I inizialize this empty list to store the indices of of s in which the symbols of t appear as a subsequence of s

    
    # first we need to iterate over each character in t (the subsequence we are searching for)
    for i in range(len(t)):
        for j in range(position, len(s)): # then we search for t[i] in s starting from position = 0 (which will later be updated)
            if t[i] == s[j]:
                indices.append(j + 1)     # if the elements are equal then we append the idx (and add 1 for 1-based indexing in biological sequences)
                position = j + 1          # then we update our position to the next character in s
                break                     # the last step is to break the inner loop and move to the next character in t

    return ' '.join(map(str, indices)) # to make the output as the one on rosalind we return the indices as a space-separated string
 
input_file = "rosalind_sseq.txt"
result = SSEQ(input_file)
print(result)

# LCSQ exercise 

Given: Two DNA strings s and t (each having length at most 1 kbp) in FASTA format.

Return: A longest common subsequence of s and t.
        (If more than one solution exists, you may return any one.)

In [ ]:
from Bio import SeqIO

def filereader(input_file):
    sequences = list(SeqIO.parse(input_file, "fasta"))
    s = str(sequences[0].seq)
    t = str(sequences[1].seq)
    return s, t

# we want to find the longest common subsequence of s and t
def LCSQ(s, t):
    len_s, len_t = len(s), len(t)

    # first we create a matrix/table to store the LCS lengths
    table = [[0] * (len_t + 1) for _ in range(len_s + 1)]  # (len_s + 1) rows, (len_t + 1) columns, +1 to account for empty substrings

    # then we need to fill the table by looping through each character in both sequences
    for row in range(1, len_s + 1):  
        for col in range(1, len_t + 1):  
            if s[row - 1] == t[col - 1]:                                        # if characters match at position (row-1, col-1)
                table[row][col] = table[row - 1][col - 1] + 1                   # we extend LCS by 1 (diagonal + 1)
            else:  
                table[row][col] = max(table[row - 1][col], table[row][col - 1]) # if characters don't match, we take the max of the left or top value

    # then we backtrack to construct the actual LCS string
    lcs = []  # to store the LCS characters
    row, col = len_s, len_t  


    while row > 0 and col > 0:          # we start from the bottom-right of the table and continue until we reach the top or left edge
        if s[row - 1] == t[col - 1]:    # if the characters match
            lcs.append(s[row - 1])      # we add the character lcs result
            row -= 1                    # then we move diagonally up-left
            col -= 1
        elif table[row - 1][col] >= table[row][col - 1]:  # if there's no match, if the LCS length above is greater or equal
            row -= 1                                      # we move up in the table
        else:                                             # else, we move left in the table
            col -= 1

    # the last step is to reverse the result since we constructed it backward
    return ''.join(reversed(lcs))

input_file = "rosalind_lcsq.txt"  
s, t = filereader(input_file)  
result = LCSQ(s, t)  
print(result)

# EDIT exercise 

Given: Two protein strings s and t in FASTA format (each of length at most 1000 aa).

Return: The edit distance dE(s,t).

In [ ]:
from Bio import SeqIO 

def filereader(input_file):

    records = list(SeqIO.parse(input_file, "fasta")) 
    s = str(records[0].seq) 
    t = str(records[1].seq)  
    return s, t

# we compute the minimum edit distance (number of insertions, deletions, substitutions)
# needed to transform string s into string t

def EDIT(s, t):
    m, n = len(s), len(t)  

    # we create a matrix/table to store the edit distances between substrings of s and t
    table = [[0] * (n + 1) for x in range(m + 1)]

    # then we initialize the table for the base cases:
    # if t is empty, all characters in s need to be deleted
    for i in range(m + 1):
        table[i][0] = i  
    # if s is empty, all characters in t need to be inserted
    for j in range(n + 1):
        table[0][j] = j  

    # we fill the table by looping through each character in s (rows) and t (columns)
    for i in range(1, m + 1):                        # we iterate over characters in s
        for j in range(1, n + 1):                    # we iterate over characters in t
            if s[i - 1] == t[j - 1]:                 # if the characters match, no operation is needed
                table[i][j] = table[i - 1][j - 1]
            else:
                table[i][j] = min(
                    table[i - 1][j] + 1,  # deletion
                    table[i][j - 1] + 1,  # insertion
                    table[i - 1][j - 1] + 1  # substitution
                )
                # otherwise, we take the minimum of the following operations:
                # deletion: we remove a character from s 
                # insertion: we add a character to s to match t 
                # substitution: we change a character in s to match t 

    return table[m][n]     # the final value in the table is the minimum edit distance

input_file = "rosalind_edit.txt"  
s, t = filereader(input_file)  
result = EDIT(s, t) 
print(result) 

# EDTA exercise 

Given: Two protein strings s and t in FASTA format (with each string having length at most 1000 aa).

Return: The edit distance dE(s,t) followed by two augmented strings s' and t' representing an optimal alignment of sand t.

In [ ]:
from Bio import SeqIO  

def filereader(input_file):
    records = list(SeqIO.parse(input_file, "fasta"))
    s = str(records[0].seq)  
    t = str(records[1].seq)
    return s, t

def EDTA(s, t):
    m, n = len(s), len(t)  

    # we first create a table to store the edit distances 
    table = [[0] * (n + 1) for x in range(m + 1)]

    # then we initialize the table for the base cases
    for i in range(m + 1):
        table[i][0] = i  # if t is empty, delete all characters in s
    for j in range(n + 1):
        table[0][j] = j  # if s is empty, insert all characters from t

    # then we fill the table with the minimum edit distances
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s[i - 1] == t[j - 1]:                  # if the characters match no modification is needed
                table[i][j] = table[i - 1][j - 1]
            else:
                table[i][j] = min(                     
                    table[i - 1][j] + 1,  # deletion
                    table[i][j - 1] + 1,  # insertion
                    table[i - 1][j - 1] + 1  # substitution
                )
                # otherwise, we take the minimum of the following operations:
                # deletion: we remove a character from s 
                # insertion: we add a character to s to match t 
                # substitution: we change a character in s to match t 

    # then we backtrack to find the alignment of the each string
    align_s = []
    align_t = []
    i, j = m, n # we start from the bottom-right of the table

    while i > 0 or j > 0:
        if i > 0 and j > 0 and s[i - 1] == t[j - 1]:           # if characters match no operation needed, we just add them to the alignment
            align_s.append(s[i - 1])
            align_t.append(t[j - 1])
            i -= 1                                             # we move diagonally up-left
            j -= 1
        elif i > 0 and table[i - 1][j] + 1 == table[i][j]:     # if we are moving up (deletion)
            align_s.append(s[i - 1])                           # we add the character from s to align_s
            align_t.append('-')                                # and we add a gap '-' to align_t (because this character in t is missing)
            i -= 1                                             # we move up (deleting a character from s)
        elif j > 0 and table[i][j - 1] + 1 == table[i][j]:     # if we are moving left (insertion)
            align_s.append('-')                                # we add a gap '-' to align_s (because this character in s is missing)
            align_t.append(t[j - 1])                           # and we add the character from t to align_t
            j -= 1                                             # we move left (inserting a character into s)
        else:                         # substitution
            align_s.append(s[i - 1])  # we add the character from s to the alignment 
            align_t.append(t[j - 1])  # we add the character from t to the alignment
            i -= 1                    # move diagonally up-left
            j -= 1


        
    # the last step is to reverse the aligned strings because we backtracked to make them 
    align_s.reverse()
    align_t.reverse()

    return table[m][n], ''.join(align_s), ''.join(align_t)

input_file = "rosalind_edta.txt"  
s, t = filereader(input_file) 
result, align_s, align_t = EDTA(s, t)  
print(result)  # the edit distance
print(align_s)  # the aligned string s'
print(align_t)  # the aligned string t'

# CTEA exercise 

Given: Two protein strings s and t in FASTA format, each of length at most 1000 aa.

Return: The total number of optimal alignments of s and t with respect to edit alignment score, modulo 134,217,727 (227-1).

In [ ]:
from Bio import SeqIO

def filereader(input_file):
    records = list(SeqIO.parse(input_file, "fasta"))
    s = str(records[0].seq)  
    t = str(records[1].seq) 
    return s, t

def CTEA(s, t):
    modulo = 134217727
    m, n = len(s), len(t)

    table = [[0] * (n + 1) for x in range(m + 1)]         # we inizialize table to store the minimum edit distance at each position
    counter_ways = [[0] * (n + 1) for x in range(m + 1)]  # we inizialize counter_ways to store the number of ways to achieve the minimum edit distance
    
    # then we inizialize them for the base cases
    for i in range(m + 1):           # if t is empty to convert s into t we have to delete all characters in s
        table[i][0] = i              # the number of deletions is i
        counter_ways[i][0] = 1       # and there's only one way to achieve this
    for j in range(n + 1):           # if s is empty we need to insert all characters in t
        table[0][j] = j              # the number of insertion is j
        counter_ways[0][j] = 1       # and there only one way as well to achieve this
     
    # firstly we need to compute the substitution cost
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s[i - 1] == t[j - 1]:
                amount = 0  # if characters are equal no substitution is needed 
            else:
                amount = 1  # if the characters are different the cost of the substitution is 1
            
            # now we need to compute the minimum cost to achieve the conversion between substitution, deletion and insertion
            minimum = min(table[i - 1][j - 1] + amount, table[i - 1][j] + 1, table[i][j - 1] + 1)
            table[i][j] = minimum # then we store the minimum cost in the table
            
            # then we have to count the number of ways to achieve this minimum cost
            if table[i][j] == table[i - 1][j - 1] + amount:                # if substitution was used
                counter_ways[i][j] += counter_ways[i - 1][j - 1]
            if table[i][j] == table[i - 1][j] + 1:                         # if deletion was used
                counter_ways[i][j] += counter_ways[i - 1][j]
            if table[i][j] == table[i][j - 1] + 1:                         # if insertion was used
                counter_ways[i][j] += counter_ways[i][j - 1]
            
            counter_ways[i][j] %= modulo        # this is the result modulo 134217727 as asked by rosalind
    
    
    return counter_ways[m][n]     # last step is to return the total number of optimal alignments
                                  # the value at counter_ways[m][n] gives the number of ways to achieve
                                  # the minimum edit distance between the strings s and t


input_file = "rosalind_ctea.txt"
s, t = filereader(input_file)
    
result = CTEA(s, t)
print(result)

# GLOB exercise 

Given: Two protein strings s and t in FASTA format (each of length at most 1000 aa).

Return: The maximum alignment score between s and t

In [ ]:
from Bio import SeqIO

sub_matrix = [
    [4, 0, -2, -1, -2, 0, -2, -1, -1, -1, -1, -2, -1, -1, -1, 1, 0, 0, -3, -2],
    [0, 9, -3, -4, -2, -3, -3, -1, -3, -1, -1, -3, -3, -3, -3, -1, -1, -1, -2, -2],
    [-2, -3, 6, 2, -3, -1, -1, -3, -1, -4, -3, 1, -1, 0, -2, 0, -1, -3, -4, -3],
    [-1, -4, 2, 5, -3, -2, 0, -3, 1, -3, -2, 0, -1, 2, 0, 0, -1, -2, -3, -2],
    [-2, -2, -3, -3, 6, -3, -1, 0, -3, 0, 0, -3, -4, -3, -3, -2, -2, -1, 1, 3],
    [0, -3, -1, -2, -3, 6, -2, -4, -2, -4, -3, 0, -2, -2, -2, 0, -2, -3, -2, -3],
    [-2, -3, -1, 0, -1, -2, 8, -3, -1, -3, -2, 1, -2, 0, 0, -1, -2, -3, -2, 2],
    [-1, -1, -3, -3, 0, -4, -3, 4, -3, 2, 1, -3, -3, -3, -3, -2, -1, 3, -3, -1],
    [-1, -3, -1, 1, -3, -2, -1, -3, 5, -2, -1, 0, -1, 1, 2, 0, -1, -2, -3, -2],
    [-1, -1, -4, -3, 0, -4, -3, 2, -2, 4, 2, -3, -3, -2, -2, -2, -1, 1, -2, -1],
    [-1, -1, -3, -2, 0, -3, -2, 1, -1, 2, 5, -2, -2, 0, -1, -1, -1, 1, -1, -1],
    [-2, -3, 1, 0, -3, 0, 1, -3, 0, -3, -2, 6, -2, 0, 0, 1, 0, -3, -4, -2],
    [-1, -3, -1, -1, -4, -2, -2, -3, -1, -3, -2, -2, 7, -1, -2, -1, -1, -2, -4, -3],
    [-1, -3, 0, 2, -3, -2, 0, -3, 1, -2, 0, 0, -1, 5, 1, 0, -1, -2, -2, -1],
    [-1, -3, -2, 0, -3, -2, 0, -3, 2, -2, -1, 0, -2, 1, 5, -1, -1, -3, -3, -2],
    [1, -1, 0, 0, -2, 0, -1, -2, 0, -2, -1, 1, -1, 0, -1, 4, 1, -2, -3, -2],
    [0, -1, -1, -1, -2, -2, -2, -1, -1, -1, -1, 0, -1, -1, -1, 1, 5, 0, -2, -2],
    [0, -1, -3, -2, -1, -3, -3, 3, -2, 1, 1, -3, -2, -2, -3, -2, 0, 4, -3, -1],
    [-3, -2, -4, -3, 1, -2, -2, -3, -3, -2, -1, -4, -4, -2, -3, -3, -2, -3, 11, 2],
    [-2, -2, -3, -2, 3, -3, 2, -1, -2, -1, -1, -2, -3, -1, -2, -2, -2, -1, 2, 7]
]

# the one above is a substitution matrix for scoring the alignment of amino acids

aminoacids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
# this is a list containing the 20 standard amino acids, represented by their one-letter codes
indices = {aa: idx for idx, aa in enumerate(aminoacids)} #  the indices give the row and col in the sub_matrix
gap_pen = -5 #gap penalty given by the problem

def filereader(input_file):
    records = list(SeqIO.parse(input_file, "fasta"))
    s = str(records[0].seq)  
    t = str(records[1].seq) 
    return s, t

# we define a function to compute the optimal alignment score
def GLOB(s, t):
    len_s = len(s)
    len_t = len(t)
    
    table = [[0] * (len_t + 1) for x in range(len_s + 1)]
    
    # firstly we initialize the first row and first column with gap penalties
    for i in range(1, len_s + 1):
        table[i][0] = table[i - 1][0] + gap_pen # these are the gap penalties for the s sequence
    for j in range(1, len_t + 1):
        table[0][j] = table[0][j - 1] + gap_pen # these are the gap penalties for the t sequence
    
    # then we fill the rest of the table by comparing all pairs of characters from s and t
    for i in range(1, len_s + 1):
        for j in range(1, len_t + 1):
            score = sub_matrix[indices[s[i - 1]]][indices[t[j - 1]]] # we get the substitution score for aligning s[i-1] with t[j-1] from the matrix
            
            # then we calculate maximum score considering match/mismatch or gaps
            table[i][j] = max(
                table[i - 1][j - 1] + score,  # if there's a match/mismatch, substitution
                table[i - 1][j] + gap_pen,    # if there's an insertion of a gap in sequence t (deletion in sequence s)
                table[i][j - 1] + gap_pen     # if there's an insertion of a gap in sequence s (deletion in sequence t)
            )
    
    # the last step is to return the bottom-right value, which is the optimal score
    return table[len_s][len_t]


input_file = "rosalind_glob.txt"  
s, t = filereader(input_file)
result = GLOB(s, t)
print(result)

In [15]:
from random import randint, choice

# Map Reduce

In the part of the assignment you are requested to use Map Reduce paradigm to solve the following exercises.

**NOTE THAT**: **A solution that does not use map reduce is not valid!**

# Exercise 1

You have a list of dictionaries, each representing a student with the following properties: a name and an array of test scores. Your task is to use map, filter, and reduce to calculate the average test score for each student, and then return a list of dictionaries containing only the students whose average score is above 90.

In [10]:
students = [
    {"name": "Alice", "scores": [95, 92, 88, 100]},
    {"name": "Bob", "scores": [78, 81, 85, 80]},
    {"name": "Charlie", "scores": [99, 91, 94, 96]},
    {"name": "Diana", "scores": [85, 87, 89, 83]}
]

Use `map`, `reduce` and `filter` that produce an output like:

In [11]:
[
    {"name": "Alice", "average_score": 93.75},
    {"name": "Charlie", "average_score": 95.0}
]

[{'name': 'Alice', 'average_score': 93.75},
 {'name': 'Charlie', 'average_score': 95.0}]

### Test
Test your solution using the dataset generated by the following function.

In [ ]:
def generate_random_student_dataset(num_students=50):
    names = [f"Student {i}" for i in range(1, num_students + 1)]
    dataset = [
        {
            "name": name,
            "scores": [randint(50, 100) for _ in range(randint(3, 6))]  # Random scores between 50 and 100
        }
        for name in names
    ]
    return dataset

random_student_dataset = generate_random_student_dataset(50)
random_student_dataset[:3]

[{'name': 'Student 1', 'scores': [87, 100, 94]},
 {'name': 'Student 2', 'scores': [76, 88, 85]},
 {'name': 'Student 3', 'scores': [58, 80, 66]}]

In [12]:
# your code goes here

import functools
def average(student):
    tot_score = functools.reduce(lambda x, y: x + y, student['scores'])
    average_score = tot_score / len(student['scores'])
    return  { "name": student["name"], "average_score": average_score }

students_average = list(map(average, students))

average_score = filter(lambda student: student["average_score"] > 90, students_average)

list(average_score)


[{'name': 'Alice', 'average_score': 93.75},
 {'name': 'Charlie', 'average_score': 95.0}]

## Exercise 2

You have a list of dictionaries, each representing a product with the following properties: name, price, and category. Using the functions `map`, `filter`, and `reduce`, calculate the average price of the products in each category and return a list of dictionaries containing only the categories where the average price exceeds 50.

Example input:

In [13]:
products = [
    {"name": "Product A", "price": 60, "category": "Electronics"},
    {"name": "Product B", "price": 40, "category": "Electronics"},
    {"name": "Product C", "price": 70, "category": "Home"},
    {"name": "Product D", "price": 30, "category": "Home"},
    {"name": "Product E", "price": 90, "category": "Sports"}
]

Use `map`, `reduce` and `filter` that produce an output like:

In [ ]:
[
    {"category": "Electronics", "average_price": 50.0},
    {"category": "Sports", "average_price": 90.0}
]

[{'category': 'Electronics', 'average_price': 50.0},
 {'category': 'Sports', 'average_price': 90.0}]

### Test
Test your solution using the dataset generated by the following function.

In [16]:
def generate_random_product_dataset(num_products=100):
    categories = ["Electronics", "Home", "Sports", "Books", "Clothing", "Toys"]
    dataset = [
        {
            "name": f"Product {i}",
            "price": randint(10, 200),  # Random price between 10 and 200
            "category": choice(categories),  # Randomly choose a category
        }
        for i in range(1, num_products + 1)
    ]
    return dataset

# Example of using the function
random_dataset = generate_random_product_dataset(100)
random_dataset[:5]  # Display the first 5 entries to check the dataset structure


[{'name': 'Product 1', 'price': 64, 'category': 'Sports'},
 {'name': 'Product 2', 'price': 196, 'category': 'Electronics'},
 {'name': 'Product 3', 'price': 154, 'category': 'Books'},
 {'name': 'Product 4', 'price': 44, 'category': 'Clothing'},
 {'name': 'Product 5', 'price': 116, 'category': 'Home'}]

In [17]:
# your code goes here
# hints: 1) Group products by category (you don't need to use map reduce for this part), then 2) use map reduce paradigm to
# calculate the average price for each category and filter categories with an average price > 50
import functools

def categories (new_dict, product):
    category = product["category"]
    if category not in new_dict:
        new_dict[category] = []
    new_dict[category].append(product)
    return new_dict

groups_cat = functools.reduce(categories, products, {})

def average(category):
    products_in_category = groups_cat[category]
    tot_price = functools.reduce(lambda x, y: x + y["price"], products_in_category, 0)
    average_price = tot_price / len(products_in_category)
    return {"category": category, "average_price": average_price}

averages = list(map(average, groups_cat))

list(filter(lambda x: x["average_price"] > 50, averages))


[{'category': 'Sports', 'average_price': 90.0}]

# Exercise 3

You have a list of dictionaries, each representing an employee with the following properties: name, salary, and department. Your task is to use `map`, `filter`, and `reduce` to calculate the average salary for each department and return a list of dictionaries containing only the departments where the average salary is above 65,000.

**Example Input**

In [18]:
employees = [
    {"name": "John", "salary": 70000, "department": "Engineering"},
    {"name": "Jane", "salary": 75000, "department": "Engineering"},
    {"name": "Alice", "salary": 60000, "department": "HR"},
    {"name": "Bob", "salary": 68000, "department": "HR"},
    {"name": "Charlie", "salary": 90000, "department": "Marketing"},
    {"name": "Diana", "salary": 50000, "department": "Marketing"}
]

Use `map`, `reduce` and `filter` that produce an output like:

In [19]:
[
    {"department": "Engineering", "average_salary": 72500.0},
    {"department": "Marketing", "average_salary": 70000.0}
]

[{'department': 'Engineering', 'average_salary': 72500.0},
 {'department': 'Marketing', 'average_salary': 70000.0}]

### Test

Test your solution using the dataset generated by the following function.

In [21]:
def generate_random_employee_dataset(num_employees=50):
    departments = ["Engineering", "HR", "Marketing", "Sales", "Finance", "IT"]
    dataset = [
        {
            "name": f"Employee {i}",
            "salary": randint(40000, 120000),  # Random salary between 40,000 and 120,000
            "department": choice(departments)  # Randomly choose a department
        }
        for i in range(1, num_employees + 1)
    ]
    return dataset

random_employee_dataset = generate_random_employee_dataset(50)

random_employee_dataset[:3]  # Display the first 3 entries of each dataset for checking


[{'name': 'Employee 1', 'salary': 57823, 'department': 'Marketing'},
 {'name': 'Employee 2', 'salary': 79367, 'department': 'Marketing'},
 {'name': 'Employee 3', 'salary': 114713, 'department': 'Sales'}]

In [22]:
# your code goes here
# hints: 1) Group employees' salaries by department (you don't need to use map reduce for this part), then 2) use map reduce paradigm to
# calculate the average salary for each department and filter departments with an average salary > threshold
import functools

def departments(new_dict, employee):
    department = employee["department"]
    if department not in new_dict:
        new_dict[department] = []
    new_dict[department].append(employee)
    return new_dict 

groups_dep = functools.reduce(departments, employees, {})

def average(department):
    employees_in_dep = groups_dep[department]
    tot_salary = functools.reduce(lambda x, y: x + y["salary"], employees_in_dep, 0)
    average_salary = tot_salary / len(employees_in_dep)
    return {"department": department, "average_salary": average_salary}

averages = list(map(average, groups_dep))

list(filter(lambda x: x["average_salary"] > 65000.0, averages))

[{'department': 'Engineering', 'average_salary': 72500.0},
 {'department': 'Marketing', 'average_salary': 70000.0}]

# Biopython

Write the following five functions to analyze global alignments between two sequences using Biopython's `pairwise2` module:

1. **countMatches(s1, s2)**  
   This function takes two sequences (`s1`, `s2`) aligned using global alignment (pairwise2.globalxx) of the same length. It returns the number of positions where the elements of both sequences match.

2. **countMismatches(s1, s2)**  
   This function takes two sequences (`s1`, `s2`) aligned using global alignment of the same length. It returns the number of positions where the elements of the two sequences are different (i.e., they are not gaps, and the characters do not match).

3. **countGapOpens(s1, s2)**  
   This function takes two sequences (`s1`, `s2`) aligned using global alignment of the same length. It returns the number of gap openings in the alignment (a gap is opened when a '-' appears in the sequence).

4. **countGapExtensions(s1, s2)**  
   This function takes two sequences (`s1`, `s2`) aligned using global alignment of the same length. It returns the number of gap extensions (where '-' continues in the alignment after an initial gap is opened).

5. **getScore(s1, s2, matchScore, mismatchPenalty, gapOpenPenalty, gapExtensionPenalty)**  
   This function takes two sequences (`s1`, `s2`) aligned using global alignment and returns the alignment score based on the provided scoring scheme: `matchScore` for matches, `mismatchPenalty` for mismatches, `gapOpenPenalty` for opening a gap, and `gapExtensionPenalty` for extending a gap.

In [ ]:
# Add your functions here

from Bio import pairwise2
from Bio import SeqIO

def countMatches(s1, s2):
    matches = 0 
    for i in range(len(s1)):
        if s1[i] == s2[i] and s1[i] != '-' and s2[i] != '-':
            matches += 1
    return matches

def countMismatches(s1, s2):
    mismatches = 0 
    for i in range(len(s1)):
        if s1[i] != s2[i] and s1[i] != '-' and s2[i] != '-':
            mismatches += 1
    return mismatches

def countGapOpens(s1, s2):
    gap_opens = 0
    if s1[0] == '-':
        gap_opens += 1
    if s2[0] == '-':
        gap_opens += 1
    for i in range(1, len(s1)):  
        if s1[i] == '-' and s1[i-1] != '-':
            gap_opens += 1
        if s2[i] == '-' and s2[i-1] != '-':
            gap_opens += 1
    return gap_opens

def countGapExtensions(s1, s2):
    gap_extensions = 0
    for i in range(1, len(s1)):
        if s1[i] == '-' and s1[i-1] == '-':
            gap_extensions += 1
        if s2[i] == '-' and s2[i-1] == '-':
            gap_extensions += 1
    return gap_extensions

def getScore(s1, s2, matchScore, mismatchPenalty, gapOpenPenalty, gapExtensionPenalty):
    matches = countMatches(s1, s2)
    mismatches = countMismatches(s1, s2)
    gap_opens = countGapOpens(s1, s2)
    gap_extensions = countGapExtensions(s1, s2)

    score = 0   
    score += matches * matchScore
    score += mismatches * mismatchPenalty
    score += gap_opens * gapOpenPenalty
    score += gap_extensions * gapExtensionPenalty
    return score

### Test
Align the sequences of the [Interleukin-12](https://en.wikipedia.org/wiki/Interleukin_12) chain A (denoted as `s1`) from the file [`IL12A.fasta`](https://qcbsciprolab2020.readthedocs.io/en/latest/file_samples/IL12A.fasta) and the Interleukin-12 chain B (denoted as `s2`) from the file [`IL12B.fasta`](https://qcbsciprolab2020.readthedocs.io/en/latest/file_samples/IL12B.fasta) and check the score as computed from pairwise2 and from your functions.

In [ ]:
# add the output of the test here

from Bio import pairwise2
from Bio import SeqIO

record = SeqIO.read("IL12A.fasta", "fasta")
s1 =  record.seq

record2 = SeqIO.read("IL12B.fasta", "fasta")
s2 = record2.seq

alignments = pairwise2.align.globalms(s1, s2, 1, -1, -0.5, -0.2)
alignment = alignments[0]
print(alignment.score) # 3.3999999999999844

s1, s2 = alignments[0][0], alignments[0][1]
print(getScore(s1, s2, 1, -1, -0.5, -0.2)) # 3.3999999999999986